# VegET Testing

Imports

In [1]:
import VegET
from VegET import interpolate, daily_aggregate, utils, veg_et_model
#import cartoee as cee
import matplotlib.pyplot as plt
import ee
import ee.mapclient
from ipygee import *

Initialize EarthEngine

In [2]:
ee.Initialize()

Define date range

In [3]:
start_date = ee.Date('2003-04-01')
end_date = ee.Date('2003-11-01')

Define ROI 

In [4]:
# ROI
roi_fc = ee.FeatureCollection('EPA/Ecoregions/2013/L4');
polygon = roi_fc.filter(ee.Filter.eq('system:index', '00000a53e3e196f3200c'))

# Filter to only include images within the colorado and utah boundaries (from ee-api/python examples)
# polygon = ee.Geometry.Polygon([[
#     [-109.05, 37.0], [-102.05, 37.0], [-102.05, 41.0],   # colorado
#     [-109.05, 41.0], [-111.05, 41.0], [-111.05, 42.0],   # utah
#     [-114.05, 42.0], [-114.05, 37.0], [-109.05, 37.0]]])

Define growing season months as integers. Note, filtering is inclusive.

In [5]:
g_season_begin = 4
g_season_end = 10

**NOTE**: for this case, the imagecollections are global or continent wide rasters. Ordinarily, the
imageCollections would need `.filterBounds()` to the ROI to subset to the images that intersect the
polygon. In this case, the filter does nothing since the images are continent/global scale.

Get ImageCollection used to calculate NDVI values. In this example, MODIS data are used.

In [6]:
ndvi_coll = ee.ImageCollection("MODIS/006/MOD09Q1").filterDate(start_date, end_date)\
    .filter(ee.Filter.calendarRange(g_season_begin, g_season_end, 'month'))\
    .map(lambda f: f.clip(polygon))
ndvi_coll = ndvi_coll.map(VegET.utils.getNDVI)

Get daily climate data (precip, eto, temp)

In [7]:
precip_eto_coll = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET').filterDate(start_date, end_date)\
    .select('pr', 'eto', 'tmmn', 'tmmx').filter(ee.Filter.calendarRange(g_season_begin, g_season_end, 'month'))\
    .map(lambda f: f.clip(polygon))

# Add band for calculated mean daily temp
precip_eto_coll = precip_eto_coll.map(VegET.utils.dailyMeanTemp)
# Convert to Celsius
precip_eto_coll = precip_eto_coll.map(VegET.utils.kelvin2celsius).select(['pr', 'eto', 'tminC', 'tmaxC', 'tmeanC'])

VegET static inputs

In [8]:
# Specify canopy intercept image or imageCollection. NOTE: Assumes single band image
canopy_int = ee.Image('users/darin_EE/VegET/Interception').clip(polygon).double().rename('intercept')
# Get static Soil Water Holding Capacity grid (manually uploaded as GEE asset)
whc = ee.Image('users/darin_EE/VegET/WaterHoldingCapacity_mm').clip(polygon).double().rename('whc')
# Get static Soil Saturation image
soil_sat = ee.Image('users/darin_EE/VegET/SoilSaturation_mm').clip(polygon).double().rename('soil_sat')
# Get static Field Capacity image
fcap = ee.Image('users/darin_EE/VegET/FieldCapacity_mm').clip(polygon).double().rename('fcap')

# Create single static image with static inputs as bands
staticImage = canopy_int.addBands([whc, soil_sat, fcap])

Add static data to ndvi_coll as bands

In [9]:
ndvi_coll = ndvi_coll.map(VegET.utils.addStaticBands([staticImage]))

Daily interpolation. Primarily using methods developed in OpenET

In [10]:
# Create daily interpolated ndvi collection
ndvi_daily = interpolate.daily(precip_eto_coll, ndvi_coll)

# Add date band as 'time'
ndvi_daily = ee.ImageCollection(ndvi_daily.map(VegET.utils.add_date_band))

## Run VegET model

In [11]:
vegET_run = veg_et_model.vegET_model(ndvi_daily, polygon)

In [ ]:
image = vegET_run.first()
print(image.bandNames().getInfo())
print(image.getInfo())

View results

## Create a Map instance    

Arguments:   
- tabs: a tuple indicating which tabs to load in the map. Options are: Inspector, Layers, Assets, Tasks   
- kwargs: as this class inherits from ipyleaflet. Map it can accept all its arguments

In [ ]:
Map = Map()

## Show map with method show
- Arguments
    - tabs: show tabs (bool)
    - layer_control: show a control for layers (bool)
    - draw_control: show a control for drawings (bool)
    - fullscrean: show fullscreen button (bool)

In [ ]:
Map.show()

## Resize Map
Dimensions must be in pixel units

In [ ]:
Map.setDimensions('90%', '500px')

## Define visualization parameters

In [ ]:
visParam = {'bands': ['swi', 'swe', 'snowpack'], 'min': 0, 'max': 100}
visParamNDVI = {'bands': ['ndvi'], 'min': 0, 'max': 1}

## Add Layers   

In [ ]:
Map.addLayer(image, visParam, name = 'VegET first')
#Map.addLayer(polygon, name = 'ROI')

## Add multiple images from a collection   

In [ ]:
Map.addImageCollection(vegET_run.limit(11), {}, namePattern = 'VegET results for {system_date}')

## Timeseries plots   

In [19]:
test_site = ee.Geometry.Point([-102.19735021056552, 46.29096881904225])
test_feat = ee.Feature(test_site, {'name': 'test feature', 'buffer': 0})
bands = ['swf']

testColl = vegET_run.filterDate('2003-06-10', '2003-08-30')

chart_ts = chart.Image.series(**{
    'imageCollection': testColl,
    'region': test_feat,
    'scale': 250,
    'bands': bands,
    'label_bands': ['swf']#,
#     'properties': ['system:index'],
#     'label_properties': ['index']
})

In [20]:
chart_ts.renderWidget(width = '75%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [21]:
chart_ts.dataframe

,swf
2003-06-10 06:00:00,22.250900
2003-06-11 06:00:00,21.272369
2003-06-12 06:00:00,18.652872
2003-06-13 06:00:00,16.129505
2003-06-14 06:00:00,13.747490
2003-06-15 06:00:00,12.043420
2003-06-16 06:00:00,10.766823
2003-06-17 06:00:00,9.399175
2003-06-18 06:00:00,8.104948
2003-06-19 06:00:00,6.757734
